# Setup

In [ ]:
# For TPU
#import os
#assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl
##!curl https://raw.githubusercontent.com/pytorch/xla/v1.7.0/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
##!python pytorch-xla-env-setup.py --version 1.7 --apt-packages libomp5 libopenblas-dev

#!pip install git+https://github.com/PytorchLightning/pytorch-lightning.git@master --upgrade
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!pip install pytorch-lightning==1.1.3 audiomentations
!git clone https://github.com/vjoki/fsl-experi.git || ( cd fsl-experi/ ; git pull )

## Copy and extract datasets from Drive

In [ ]:
![ ! -d fsl-experi/data/LibriSpeech/ ] && cp ./drive/MyDrive/colab/train-clean-100.tar.gz ./drive/MyDrive/colab/test-clean.tar.gz ./drive/MyDrive/colab/dev-clean.tar.gz . \
  && tar zxf ./train-clean-100.tar.gz -C fsl-experi/data/ \
  && tar zxf ./dev-clean.tar.gz -C fsl-experi/data/ \
  && tar zxf ./test-clean.tar.gz -C fsl-experi/data/
![ ! -d fsl-experi/data/RIRS_NOISES/ ] && cp ./drive/MyDrive/colab/rirs_noises.zip . && unzip -q -d fsl-experi/data/ rirs_noises.zip
!mkdir ./fsl-experi/lightning_logs/

# Train

In [ ]:
# Start tensorboard.
%reload_ext tensorboard
%tensorboard --logdir ./fsl-experi/lightning_logs/

In [ ]:
!cd fsl-experi/ && python -m snn.librispeech.train --gpus 1 --num_workers 4  \
  --max_epochs 150 --batch_size 64 --max_sample_length 4 \
  --resnet_aggregation_type ASP --n_mels 64 --model snn
  #--train_batch_size 1 --num_ways 32 --num_shots 2 \
  #--num_speakers 50 \

# Save model

In [ ]:
!cp ./fsl-experi/checkpoints/*.ckpt ./drive/MyDrive/colab/checkpoints/
!cp -r ./fsl-experi/lightning_logs/ ./drive/MyDrive/colab/